In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [109]:
df = pd.read_csv('fraud_data.csv')
df.head()

,trans_date_trans_time,merchant,category,amt,city,state,lat,long,city_pop,job,dob,trans_num,merch_lat,merch_long,is_fraud
0,04-01-2019 00:58,"""Stokes, Christiansen and Sipes""",grocery_net,14.37,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",09-11-1939,a3806e984cec6ac0096d8184c64ad3a1,65.654142,-164.722603,1
1,04-01-2019 15:06,Predovic Inc,shopping_net,966.11,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",09-11-1939,a59185fe1b9ccf21323f581d7477573f,65.468863,-165.473127,1
2,04-01-2019 22:37,Wisozk and Sons,misc_pos,49.61,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",09-11-1939,86ba3a888b42cd3925881fa34177b4e0,65.347667,-165.914542,1
3,04-01-2019 23:06,Murray-Smitham,grocery_pos,295.26,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",09-11-1939,3a068fe1d856f0ecedbed33e4b5f4496,64.445035,-166.080207,1
4,04-01-2019 23:59,Friesen Lt,health_fitness,18.17,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",09-11-1939,891cdd1191028759dc20dc224347a0ff,65.447094,-165.446843,1


In [111]:
#Dig out na's
np.sum(pd.isna(df))
#None to speak of

C:\Users\sgray\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


trans_date_trans_time    0
merchant                 0
category                 0
amt                      0
city                     0
state                    0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
merch_lat                0
merch_long               0
is_fraud                 0
dtype: int64

In [113]:
#Check for duplicates
df.duplicated().sum()

63

In [115]:
#There's quite a few, inspect em
first_dupe = df.loc[df.duplicated].trans_num.reset_index().trans_num[0]

df.loc[df.trans_num == first_dupe]
#Yep, these are all dupes

,trans_date_trans_time,merchant,category,amt,city,state,lat,long,city_pop,job,dob,trans_num,merch_lat,merch_long,is_fraud
0,04-01-2019 00:58,"""Stokes, Christiansen and Sipes""",grocery_net,14.37,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",09-11-1939,a3806e984cec6ac0096d8184c64ad3a1,65.654142,-164.722603,1
8725,04-01-2019 00:58,"""Stokes, Christiansen and Sipes""",grocery_net,14.37,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",09-11-1939,a3806e984cec6ac0096d8184c64ad3a1,65.654142,-164.722603,1


In [117]:
#Drop em
df = df.drop_duplicates()
df.duplicated().sum()

0

In [139]:
#Check for erroneous "is_fraud" values
df.loc[df.is_fraud != 0]
#How strange, it doesn't recognize 1 or 0 as ints
df.is_fraud.dtype #Says it's an object, not an int, let's change that
df.is_fraud = df.is_fraud.astype('int')
#ValueError: invalid literal for int() with base 10: '1"2020-12-24 16:56:24"', woah what's up

ValueError: invalid literal for int() with base 10: '1"2020-12-24 16:56:24"'

In [147]:
df.loc[df.is_fraud == '1"2020-12-24 16:56:24"'] #oopsie


,trans_date_trans_time,merchant,category,amt,city,state,lat,long,city_pop,job,dob,trans_num,merch_lat,merch_long,is_fraud
1781,11-12-2020 23:19,Thompson-Gleason,health_fitness,19.45,Lakeport,CA,39.047,-122.9328,11256,Podiatrist,18-10-1972,bfde75d978bb9905a4a8c87440692a4c,39.25188,-122.490946,"1""2020-12-24 16:56:24"""


In [155]:
#Probably supposed to be a 1
df.loc[1781, "is_fraud"] = 1
df.loc[df.is_fraud == '1"2020-12-24 16:56:24"'] #fixed, run it back

,trans_date_trans_time,merchant,category,amt,city,state,lat,long,city_pop,job,dob,trans_num,merch_lat,merch_long,is_fraud


In [159]:
df.is_fraud = df.is_fraud.astype('int')
#ValueError: invalid literal for int() with base 10: '0"2019-01-01 00:00:44"', sigh...

ValueError: invalid literal for int() with base 10: '0"2019-01-01 00:00:44"'

In [163]:
df.loc[df.is_fraud == '0"2019-01-01 00:00:44"'] #probs a 0
df.loc[7780, "is_fraud"] = 0
df.loc[df.is_fraud == '0"2019-01-01 00:00:44"']

,trans_date_trans_time,merchant,category,amt,city,state,lat,long,city_pop,job,dob,trans_num,merch_lat,merch_long,is_fraud


In [195]:
 #once more with feeling
df.is_fraud = df.is_fraud.astype('int')
#swish
print(df.shape)
print(sum(df.is_fraud == 0))
print(sum(df.is_fraud == 1))
print(sum(df.is_fraud == 0) + sum(df.is_fraud == 1))
(sum(df.is_fraud == 0) + sum(df.is_fraud == 1)) == df.shape[0]
#Everything looks good!
#df.to_csv('cleaned_data.csv')

(14383, 15)
12601
1782
14383
